In [2]:
for i in range(3):
    print(i)

0
1
2


In [3]:
t1 = (0,1)
t2 = (2)
t3= (t1,t2)
print(t3)

((0, 1), 2)


In [4]:
print("\nImports...")
import sys
sys.path.append( '/home/ebutz/ESL2024/code/utils' )
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import pickle
import os
import sys
sys.path.append( '/home/ebutz/ESL2024/code/utils' )
import play_with_complex as pwc

import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

from nxontology.imports import from_file


Imports...


/home/ebutz/miniconda3/envs/pyg/lib/python3.11/site-packages/torch_geometric/typing.py:54: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /home/ebutz/miniconda3/envs/pyg/lib/python3.11/site-packages/libpyg.so: undefined symbol: _ZNK5torch8autograd4Node4nameEv
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "


In [5]:
a = torch.tensor([0,1])
b = torch.tensor([2,3])
c = torch.cat([a,b])
print(c)

tensor([0, 1, 2, 3])


In [6]:


# ------------- Paths ------------- #

# Iric
mapped_iric_path  = '/home/ebutz/ESL2024/data/Os_to_GO_iric/altailed_Os_to_GO_iric.pickle'
altails_dict_path = '/home/ebutz/ESL2024/data/Os_to_GO_iric/DICT_altailed_Os_to_GO_iric.pickle'

# Model to train :
hidden_channels = 176
batch_size      = 4096
epochs          = 200
eval_period     = 2
lin_factor      = 1

use_wandb  = True

params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{epochs}_epochs_{batch_size}_BS_on_full_iric"
model_parameters_path = "/home/ebutz/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

# Ontology
ontology_path = "/home/ebutz/ESL2024/data/go-basic.json.gz"
check_dicts = False

# ------------- Cuda ------------- #

print("\nCuda check...")

device = 'cpu'

# ------------- Loading datas ------------- #

print("\nLoading iric...")
mapped_iric = pd.read_pickle(mapped_iric_path)
print(mapped_iric.head())
print('mapped_alt_tails type :', type(mapped_iric.iloc[0]['mapped_alt_tails']))

GO_to_map = mapped_iric.set_index('object')['mapped_object'].to_dict()
map_to_GO = {value: key for key, value in GO_to_map.items()}

# if check_dicts:
#     looks_ok: bool = True
#     for i in tqdm(range(len(list(mapped_iric['object']))), desc = "Checking GO to MAP dict"):
#         if GO_to_map[mapped_iric['object'][i]]!=mapped_iric['mapped_object'][i] :
#             looks_ok = False
#     print('GO - Mapping dicts looks ok :', looks_ok)

with open(altails_dict_path, 'rb') as handle:
    mapped_alt_tails = pickle.load(handle)
print("Alternative tails dict (first key-value pair):", list(mapped_alt_tails.items())[0])

# # ------------- Making datasets ------------- #

print("\nMaking datasets...")
# Edges index :
heads = list(mapped_iric['mapped_subject'])
tails = list(mapped_iric['mapped_object'])
edge_index = torch.tensor([heads,tails], dtype=torch.long)
# edges attributes :
edge_attributes = torch.tensor(mapped_iric['mapped_predicate'])


iric_pyg = Data(
                num_nodes = len(set(mapped_iric['object']).union(set(mapped_iric['subject']))),
                edge_index = edge_index,
                edge_attr = edge_attributes
                )

print(iric_pyg)

print("\nDataset looks valid :",iric_pyg.validate(raise_on_error=True))

transform = RandomLinkSplit(
                            num_val = 0.1,
                            num_test = 0.1,
                            is_undirected = False,
                            add_negative_train_samples = False,
                            )

train_set, val_set, test_set = transform(iric_pyg)
print('Dataset splits look valid (train, val, test):',train_set.validate(raise_on_error = True),
                                                      val_set.validate(raise_on_error   = True),
                                                      test_set.validate(raise_on_error  = True))

# ------------- Loading ontology ------------- #

# print("\nLoading ontology...")
# nxo = from_file(ontology_path)
# nxo.freeze()
# pwc.nxo              = nxo

# ------------- Making global variables accessibles to pwc ------------- #

pwc.map_to_GO        = map_to_GO
pwc.mapped_alt_tails = mapped_alt_tails
pwc.device           = device


print('All done.')



Cuda check...

Loading iric...
            subject      predicate      object  mapped_subject  \
0  OsNippo01g010050  gene ontology  GO:0031267           29269   
1  OsNippo01g010050  gene ontology  GO:0006886           29269   
2  OsNippo01g010050  gene ontology  GO:0005622           29269   
3  OsNippo01g010050  gene ontology  GO:0005623           29269   
4  OsNippo01g010050  gene ontology  GO:0090630           29269   

   mapped_predicate  mapped_object                            mapped_alt_tails  
0                 0           8105  [20179, 25142, 11107, 11397, 25543, 24490]  
1                 0          20179   [8105, 25142, 11107, 11397, 25543, 24490]  
2                 0          25142   [8105, 20179, 11107, 11397, 25543, 24490]  
3                 0          11107   [8105, 20179, 25142, 11397, 25543, 24490]  
4                 0          11397   [8105, 20179, 25142, 11107, 25543, 24490]  
mapped_alt_tails type : <class 'numpy.ndarray'>
Alternative tails dict (first key-val

In [7]:
pwc.map_to_GO        = map_to_GO
pwc.mapped_alt_tails = mapped_alt_tails
pwc.device           = device


In [9]:
to1 = pwc.tail_only_ComplEx(num_nodes                            = train_set.num_nodes,
                               num_relations                     = train_set.edge_index.size()[1],
                               hidden_channels                   = hidden_channels,
                               additionnal_negative_per_positive = 2
                               ).to(device)

loader = to1.loader(head_index = test_set.edge_index[0], tail_index = test_set.edge_index[1], rel_type   = test_set.edge_attr,batch_size = batch_size, shuffle    = True)
batchy = next(iter(loader))
print(batchy)
to1.loss(*batchy)

(tensor([ 9642,  2583, 14253,  ..., 15481,  5034, 13213]), tensor([0, 0, 0,  ..., 0, 0, 0]), tensor([17406,  5748,  4763,  ..., 24599, 13425, 10747]))
added additional negatives!
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([8192])
torch.Size([8192])
torch.Size([8192])
(tensor([ 9642,  2583, 14253,  ..., 15481,  5034, 13213]), tensor([0, 0, 0,  ..., 0, 0, 0]), tensor([17406,  5748,  4763,  ..., 24599, 13425, 10747]))
(tensor([ 9642,  2583, 14253,  ..., 15481,  5034, 13213]), tensor([0, 0, 0,  ..., 0, 0, 0]), tensor([28205, 14535, 11938,  ..., 27341, 16445, 24599]))


tensor(5.4860, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [ ]:
# print("\nImports...")

# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# tqdm.pandas()

# import pickle
# import os
# import sys
# sys.path.append( '/home/ebutz/ESL2024/code/utils' )
# import play_with_complex as pwc

# import torch
# import torch.optim as optim
# import torch_geometric
# from torch_geometric.nn import ComplEx
# from torch_geometric.data import Data
# from torch_geometric.transforms import RandomLinkSplit
# from torch_geometric.loader import DataLoader
# import torch.nn.functional as F

# from nxontology.imports import from_file

# # ------------- Paths ------------- #

# # Iric
# mapped_iric_path  = '/home/ebutz/ESL2024/data/Os_to_GO_iric/altailed_Os_to_GO_iric.pickle'
# altails_dict_path = '/home/ebutz/ESL2024/data/Os_to_GO_iric/DICT_altailed_Os_to_GO_iric.pickle'

# # Model to train :
# hidden_channels = 176
# batch_size      = 4096
# epochs          = 1000
# eval_period     = 4
# lin_factor      = 1

# use_wandb  = True

# params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{epochs}_epochs_{batch_size}_BS_on_full_iric"
# model_parameters_path = "/home/ebutz/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

# # Ontology
# ontology_path = "/home/ebutz/ESL2024/data/go-basic.json.gz"
# check_dicts = True

# # ------------- Cuda ------------- #

# print("\nCuda check...")

# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Device: '{device}'")
# print("Could reach GPU :", torch.Tensor([0,1]).to(device).is_cuda)


# # ------------- Loading datas ------------- #

# print("\nLoading iric...")
# mapped_iric = pd.read_pickle(mapped_iric_path)
# print(mapped_iric.head())
# print('mapped_alt_tails type :', type(mapped_iric.iloc[0]['mapped_alt_tails']))

# GO_to_map = mapped_iric.set_index('object')['mapped_object'].to_dict()
# map_to_GO = {value: key for key, value in GO_to_map.items()}

# if check_dicts:
#     looks_ok: bool = True
#     for i in tqdm(range(len(list(mapped_iric['object']))), desc = "Checking GO to MAP dict"):
#         if GO_to_map[mapped_iric['object'][i]]!=mapped_iric['mapped_object'][i] :
#             looks_ok = False
#     print('GO - Mapping dicts looks ok :', looks_ok)

# with open(altails_dict_path, 'rb') as handle:
#     mapped_alt_tails = pickle.load(handle)
# print("Alternative tails dict (first key-value pair):", list(mapped_alt_tails.items())[0])

# # # ------------- Making datasets ------------- #

# print("\nMaking datasets...")
# # Edges index :
# heads = list(mapped_iric['mapped_subject'])
# tails = list(mapped_iric['mapped_object'])
# edge_index = torch.tensor([heads,tails], dtype=torch.long)
# # edges attributes :
# edge_attributes = torch.tensor(mapped_iric['mapped_predicate'])


# iric_pyg = Data(
#                 num_nodes = len(set(mapped_iric['object']).union(set(mapped_iric['subject']))),
#                 edge_index = edge_index,
#                 edge_attr = edge_attributes
#                 )

# print(iric_pyg)

# print("\nDataset looks valid :",iric_pyg.validate(raise_on_error=True))

# transform = RandomLinkSplit(
#                             num_val = 0.1,
#                             num_test = 0.1,
#                             is_undirected = False,
#                             add_negative_train_samples = False,
#                             )

# train_set, val_set, test_set = transform(iric_pyg)
# print('Dataset splits look valid (train, val, test):',train_set.validate(raise_on_error = True),
#                                                       val_set.validate(raise_on_error   = True),
#                                                       test_set.validate(raise_on_error  = True))

# # ------------- Loading ontology ------------- #

# print("\nLoading ontology...")
# nxo = from_file(ontology_path)
# nxo.freeze()

# # ------------- Making global variables accessibles to pwc ------------- #

# pwc.map_to_GO        = map_to_GO
# pwc.nxo              = nxo
# pwc.mapped_alt_tails = mapped_alt_tails
# pwc.device           = device


# epochs = 2
# eval_period = 1
# # ------------- Init model ------------- #
# # 250, 15, 75, 15, 176, 250, 350
# for hidden_channels in [5]:
#     xp_name = f"Lin VS Baseline with {hidden_channels}*6 HC on {epochs} epochs"

#     pwc.train_model(model_name='tail_only_ComplEx', hidden_channels_list=[hidden_channels], epochs=epochs, eval_period=eval_period,
#                 device='cpu', use_wandb=True, xp_name=f'Lin or noise with {hidden_channels} HC',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)
    
#     pwc.train_model(model_name='ComplEx_L_labels', hidden_channels_list=[hidden_channels], epochs=epochs, eval_period=eval_period,
#                 device='cpu', use_wandb=True, xp_name=f'Lin or noise with {hidden_channels} HC',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)
    
#     pwc.train_model(model_name='ComplEx_L_FRL_labels', hidden_channels_list=[hidden_channels], epochs=epochs, eval_period=eval_period,
#                 device='cpu', use_wandb=True, xp_name=f'Lin or noise with {hidden_channels} HC',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)
    
#     pwc.train_model(model_name='ComplEx_FRL_U_labels', hidden_channels_list=[hidden_channels], epochs=epochs, eval_period=eval_period,
#                 device='cpu', use_wandb=True, xp_name=f'Lin or noise with {hidden_channels} HC',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)

#     pwc.train_model(model_name='ComplEx_UGN_labels', hidden_channels_list=[hidden_channels], epochs=epochs, eval_period=eval_period,
#                 device='cpu', use_wandb=True, xp_name=f'Lin or noise with {hidden_channels} HC',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)
    
#     pwc.train_model(model_name='ComplEx_LGN_labels', hidden_channels_list=[hidden_channels], epochs=epochs, eval_period=eval_period,
#                 device='cpu', use_wandb=True, xp_name=f'Lin or noise with {hidden_channels} HC',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)
    

# print('Finished !')











Imports...

Cuda check...
Device: 'cuda'
Could reach GPU : True

Loading iric...
            subject      predicate      object  mapped_subject  \
0  OsNippo01g010050  gene ontology  GO:0031267           29269   
1  OsNippo01g010050  gene ontology  GO:0006886           29269   
2  OsNippo01g010050  gene ontology  GO:0005622           29269   
3  OsNippo01g010050  gene ontology  GO:0005623           29269   
4  OsNippo01g010050  gene ontology  GO:0090630           29269   

   mapped_predicate  mapped_object                            mapped_alt_tails  
0                 0           8105  [20179, 25142, 11107, 11397, 25543, 24490]  
1                 0          20179   [8105, 25142, 11107, 11397, 25543, 24490]  
2                 0          25142   [8105, 20179, 11107, 11397, 25543, 24490]  
3                 0          11107   [8105, 20179, 25142, 11397, 25543, 24490]  
4                 0          11397   [8105, 20179, 25142, 11107, 25543, 24490]  
mapped_alt_tails type : <class 'num

Checking GO to MAP dict: 100%|██████████| 169248/169248 [00:01<00:00, 135224.38it/s]


GO - Mapping dicts looks ok : True
Alternative tails dict (first key-value pair): ((29269, 0), [11107, 11397, 25543, 8105, 24490, 20179, 25142])

Making datasets...
Data(edge_index=[2, 169248], edge_attr=[169248], num_nodes=30396)

Dataset looks valid : True
Dataset splits look valid (train, val, test): True True True

Loading ontology...


KeyboardInterrupt: 

In [ ]:
# pwc.train_model(model_name='tail_only_ComplEx', hidden_channels_list=[1000], epochs=150, eval_period=10,
#                 device='cpu', use_wandb=True, xp_name='1000 HC !',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)

# pwc.train_model(model_name='ComplEx_LGN_labels', hidden_channels_list=[1000], epochs=150, eval_period=5,
#                 device='cpu', use_wandb=True, xp_name='1000 HC !',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)

# pwc.train_model(model_name='ComplEx_GN_U_labels', hidden_channels_list=[1000], epochs=150, eval_period=5,
#                 device='cpu', use_wandb=True, xp_name='1000 HC !',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)

# pwc.train_model(model_name='ComplEx_L_labels', hidden_channels_list=[1000], epochs=150, eval_period=5,
#                 device='cpu', use_wandb=True, xp_name='1000 HC !',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)



# pwc.train_model(model_name='ComplEx_UGN_labels', hidden_channels_list=[1000], epochs=150, eval_period=5,
#                 device='cpu', use_wandb=True, xp_name='1000 HC !',
#                 train_set=train_set, test_set=test_set,
#                 file_import = pwc)

In [ ]:
type(train_set)

torch_geometric.data.data.Data

In [ ]:
l = [1,2,3]
l.append(4)
l

[1, 2, 3, 4]

In [ ]:
# hidden_channels = 15
# xp_name = "Full randomised labels with 15*6 HC"

# GN_U = pwc.ComplEx_GN_U_labels(num_nodes       = train_set.num_nodes,
#                                num_relations   = train_set.edge_index.size()[1],
#                                hidden_channels = hidden_channels,
#                                ).to(device)

# loader = GN_U.loader(
#                           head_index = test_set.edge_index[0],
#                           tail_index = test_set.edge_index[1],
#                           rel_type   = test_set.edge_attr,
#                           batch_size = batch_size,
#                           shuffle    = True)

# batch = next(iter(loader))
# print(batch)
# GN_U.test(*batch, k=10)

In [ ]:
# GN_U = pwc.ComplEx_GN_U_labels(num_nodes       = train_set.num_nodes,
#                                num_relations   = train_set.edge_index.size()[1],
#                                hidden_channels = hidden_channels,
#                                ).to(device)
# pwc.train_and_test_complex(model      = GN_U,
#                             train_data = train_set,
#                             test_data  = test_set,
#                             device     = device,
#                             use_wandb  = False,
#                             xp_name    = xp_name,
#                             run_name   = 'GN_U_labels',
#                             eval_period= eval_period,
#                             epochs = epochs
#                             )

In [ ]:
a = 0
for i in tqdm(range(1000)):
    i-=1

100%|██████████| 1000/1000 [00:00<00:00, 936437.60it/s]


In [ ]:
LGN = pwc.ComplEx_LGN_labels(num_nodes       = train_set.num_nodes,
                                 num_relations   = train_set.edge_index.size()[1],
                                 hidden_channels = hidden_channels,
                                 ).to(device)
pwc.train_and_test_complex(model       = LGN,
                            train_data = train_set,
                            test_data  = test_set,
                            device     = device,
                            use_wandb  = False,
                            xp_name    = 'nay',
                            run_name   = 'LGN',
                            eval_period= eval_period,
                            epochs = epochs
                            )

Train and test.
Model type : <class 'play_with_complex.ComplEx_LGN_labels'>
Hidden channels : 6 * 176
Epochs : 200
Eval period : 2
Use WanDB : False
Device : cpu Could reach GPU : False
Evaluation each 2 epochs.


AttributeError: module 'nxontology' has no attribute 'graph'

In [ ]:
hidden_channels = 15
xp_name = "Full randomised labels with 15*6 HC"



# BLS_U = pwc.ComplEx_BLS_U_labels(num_nodes       = train_set.num_nodes,
#                                  num_relations   = train_set.edge_index.size()[1],
#                                  hidden_channels = hidden_channels,
#                                  ).to(device)
# pwc.train_and_test_complex(model       = BLS_U,
#                             train_data = train_set,
#                             test_data  = test_set,
#                             device     = device,
#                             use_wandb  = False,
#                             xp_name    = xp_name,
#                             run_name   = 'BLS_U_labels',
#                             eval_period= eval_period,
#                             epochs = epochs
#                             )

# FRBLS_U = pwc.ComplEx_FRBLS_U_labels(num_nodes      = train_set.num_nodes,
#                                     num_relations   = train_set.edge_index.size()[1],
#                                     hidden_channels = hidden_channels,
#                                     ).to(device)
# pwc.train_and_test_complex(model       = FRBLS_U,
#                             train_data = train_set,
#                             test_data  = test_set,
#                             device     = device,
#                             use_wandb  = True,
#                             xp_name    = xp_name,
#                             run_name   = 'FRBLS_U_labels',
#                             eval_period= eval_period,
#                             epochs = epochs
#                             )


# print('Finished !')

In [ ]:
a = torch.tensor([0,0,12,1,23,1])
print(a.unique())

tensor([ 0,  1, 12, 23])
